In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
import csv
from selenium.webdriver.common.action_chains import ActionChains
import urllib
import xlwt
wb = xlwt.Workbook()

chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1366x768")
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_driver = 'C:\\Users\\Kumarsur\\Desktop\\python\\chromedriver'
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=chrome_driver)
driver.maximize_window()

driver.get("https://tabulator.universumglobal.com/ui/default.aspx") 
time.sleep(2)
obj = driver.switch_to.alert
time.sleep(2)
obj.dismiss()

username = driver.find_element_by_id("txtUserName")
password = driver.find_element_by_id("txtPassword")
username.send_keys("pantheon\surendra.kumar")
password.send_keys("Feb@2019")
login_attempt = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[2]/td[2]/table/tbody/tr[8]/td/table/tbody/tr/td/div/a")
login_attempt.submit()

country_list = ["CASS 2019", "CASS 2018", "CATS 2017", "CATS 2016", "CATS 2015", "CASS 2014", "CASS 2013", "CASS 2012", "CASS 2011", "CASS 2010", "CASS 2008"]
# country_list = ["CASS 2019", "CASS 2018"]


for country_name in country_list:  
    try:
        ###survey searching
        country = country_name
        x_path = "//td[contains(text(),'" + country + "')]"
        search_survey = driver.find_element_by_xpath(x_path)
        search_survey.click()
        time.sleep(1)
        ########################### FRAMES ##############################
        iframeQuestionGroups = driver.find_element_by_id('iframeQuestionGroups')
        iframeQuestions = driver.find_element_by_id('iframeQuestions')
        iframeAnalyze = driver.find_element_by_id('iframeAnalyze')
        iframeCategorize = driver.find_element_by_id('iframeCategorize')
        iframeAlternatives = driver.find_element_by_id('iframeAlternatives')
        iframeFilter = driver.find_element_by_id('iframeFilter')

        ############################# Questions Group frame###########################
        driver.switch_to.frame(iframeQuestionGroups)
        driver.implicitly_wait(30)

        if ("Official Target Groups" in driver.page_source):
            add_QuestionGroups= driver.find_element_by_xpath("//span[text() = 'Official Target Groups']") ##click Official Target Groups
            add_QuestionGroups.click()
        elif ("Educational Profile" in driver.page_source):
            add_QuestionGroups= driver.find_element_by_xpath("//span[text() = 'Educational Profile']")  ##click Educational Profile
            add_QuestionGroups.click()

        time.sleep(1)
        driver.switch_to.default_content()

        ############################# Alternatives frame double click to add FILTER ###########################
        actionChains = ActionChains(driver)

        driver.switch_to.frame(iframeAlternatives)
        driver.implicitly_wait(30)
        time.sleep(1)

        if ("Student" in driver.page_source): 
            add_filter= driver.find_element_by_xpath("//option[contains(text(), 'Student')]")  ##add student to filter 
            actionChains.double_click(add_filter).perform()
        elif ("Undergraduate" in driver.page_source): 
            add_filter= driver.find_element_by_xpath("//option[contains(text(), 'Undergraduate')]")  ##add Undergraduate to filter
            actionChains.double_click(add_filter).perform()         

        time.sleep(1)
        driver.switch_to.default_content()

        ############################# Questions Group frame ###########################
        driver.switch_to.frame(iframeQuestionGroups)
        driver.implicitly_wait(30)
        add_QuestionGroups= driver.find_element_by_xpath("//span[text() = 'Career Expectations and Preferences']")
        add_QuestionGroups.click()
        time.sleep(1)
        driver.switch_to.default_content()

        ############################# Add to Analyze ###########################
        driver.switch_to.frame(iframeAnalyze)
        driver.implicitly_wait(30)
        time.sleep(1)
        add_analyze= driver.find_element_by_xpath("//a[@onclick ='parent.AddToList(document.forms[0].listAnalyze);']")
        add_analyze.click()
        time.sleep(1)
        driver.switch_to.default_content()

        ############################# Get Table ###########################
        parent_window_handle = driver.current_window_handle
        first_window = driver.window_handles[0]

        gettable= driver.find_element_by_xpath("//a[@onclick ='getTable(false)']")
        gettable.click()

        driver.implicitly_wait(30)
        time.sleep(20)

        ############################ POPUP WINDOWs For data ###########################

        handles = driver.window_handles;

        driver.switch_to.window(handles[1])
        print(driver.title)
        print(driver.find_element_by_xpath("//a[@onclick ='saveSettings()']").text)
        print(driver.find_element_by_xpath("//a[@onclick ='execute(false)']").text)

        ############################ aply setting HTML format in popup window ###########################

        driver.implicitly_wait(30)
        time.sleep(1)

        #### Output tab
        driver.find_element_by_xpath("//em[text() = 'Output']").click()
        driver.implicitly_wait(30)
        time.sleep(1)
        driver.find_element_by_xpath("//select[@name='listOutput']/option[text()='HTML Table']").click()
        driver.implicitly_wait(30)
        time.sleep(1)
        ### General tables tab
        driver.find_element_by_xpath("//em[text() = 'General Tables']").click()
        driver.implicitly_wait(30)
        time.sleep(1)
        driver.find_element_by_xpath("//*[@id='SpanColumnPercent']/label").click()
        driver.find_element_by_xpath("//*[@id='SpanColRank']/label").click()
        driver.implicitly_wait(30)
        time.sleep(1)

        ### Apply settings tab
        driver.find_element_by_xpath("//a[text() ='Apply Settings']").click()
        driver.implicitly_wait(30)
        time.sleep(1)

        ###iframe for html data
        iframes_data = driver.find_element_by_xpath("//iframe[contains(@src, 'showData.aspx?guid')]")
        driver.switch_to.frame(iframes_data)
        driver.implicitly_wait(30)

        res = driver.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(res, 'lxml')

        ws = wb.add_sheet(country)
        table = soup.findAll('tbody')[1]  
        rows = table.findAll("tr")
        x = 0
        for tr in rows:
            cols = tr.findAll("td")
            if not cols: 
                # when we hit an empty row, we should not print anything to the workbook
                continue
            y = 0
            for td in cols:
                texte_bu = td.text
                texte_bu = texte_bu.encode('utf-8')
                texte_bu = texte_bu.strip()
                if x <= 6:
                    ws.write(x, y, td.text)
                else:
                    ws.write(x, y + 1, td.text)
                print(x, y, td.text)
                y = y + 1
            # update the row pointer AFTER a row has been printed
            # this avoids the blank row at the top of your table
            x = x + 1

        wb.save('Career goals4.xls')
        driver.switch_to.window(handles[0])
        driver.find_element_by_xpath("//a[@href ='chooseSurvey.aspx']").click()
        time.sleep(3)
    except Exception as ex:
        print(ex)
driver.quit()    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: use options instead of chrome_options


Analyze Data
Save Settings
Apply Settings
0 0 Survey: CASS 2019 (v1), 23035 cases
1 0 Weighted by: weighted total
2 0 Filter: 20676 cases, expression:((1. Official Target Groups (Also add a Main Field of Study to filter)=Students))
3 0  
4 0  
4 1 Total
5 0  
5 1 Column Total
6 0 Career goals
6 1 To be a leader or manager of people
6 2 Column Rank
6 3 5
7 0 To be a technical or functional expert
7 1 Column Rank
7 2 9
8 0 To be autonomous or independent
8 1 Column Rank
8 2 8
9 0 To be competitively or intellectually challenged
9 1 Column Rank
9 2 4
10 0 To be dedicated to a cause or to feel that I am serving a greater good
10 1 Column Rank
10 2 3
11 0 To be entrepreneurial or creative/innovative
11 1 Column Rank
11 2 6
12 0 To be secure or stable in my job
12 1 Column Rank
12 2 2
13 0 To have an international career
13 1 Column Rank
13 2 7
14 0 To have work/life balance
14 1 Column Rank
14 2 1
Analyze Data
Save Settings
Apply Settings
0 0 Survey: CASS 2018 (v1), 17310 cases
1 0 Weighted

Analyze Data
Save Settings
Apply Settings
0 0 Survey: CASS, 27983 cases
1 0 Weighted by: weighted total
2 0 No Filter
3 0  
4 0  
4 1 Total
5 0  
5 1 Column Total
6 0 Career goals
6 1 To be a leader or manager of people
6 2 Column Rank
6 3 5
7 0 To be a technical or functional expert
7 1 Column Rank
7 2 8
8 0 To be autonomous or independent
8 1 Column Rank
8 2 7
9 0 To be competitively or intellectually challenged
9 1 Column Rank
9 2 4
10 0 To be dedicated to a cause or to feel that I am serving a greater good
10 1 Column Rank
10 2 3
11 0 To be entrepreneurial or creative/innovative
11 1 Column Rank
11 2 6
12 0 To be secure or stable in my job
12 1 Column Rank
12 2 2
13 0 To have an international career
13 1 Column Rank
13 2 9
14 0 To have work/life balance
14 1 Column Rank
14 2 1
